In [1]:
import pandas as pd
import numpy as np

# library for text preprocessing
import re
import nltk

#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#nltk.download('wordnet') 


from nltk.stem.wordnet import WordNetLemmatizer

In [14]:
# selected column headers we want to use 
new_headers = ['Tags', 'CreationDate', 'LastActivityDate']
df = pd.read_csv("QueryResults_cyber.csv", header=0)
# remove unwanted columns keep selected headers
dataset = pd.DataFrame(df, columns=new_headers)

dataset

,Tags,CreationDate,LastActivityDate
0,<amazon-web-services><amazon-ec2><network-secu...,2017-06-21 06:29:34,2017-06-21 13:24:00
1,<amazon-web-services><amazon-ec2><network-secu...,2017-06-21 06:29:34,2017-06-21 13:24:00
2,<javascript><reactjs><security><jwt><rails-api>,2017-06-21 22:30:57,2017-06-21 23:07:12
3,<facebook><azure><security><azure-blob-storage>,2017-06-21 15:28:37,2017-06-22 03:22:55
4,<security>,2017-06-21 14:53:05,2017-06-22 09:55:00
...,...,...,...
33850,<php><wordpress><security><xss>,2022-06-24 17:50:25,2022-06-24 18:53:14
33851,<android><firebase-realtime-database><firebase...,2022-06-24 18:55:01,2022-06-24 19:30:32
33852,<asp.net-core><security><.net-core><ssl-certif...,2021-01-29 00:12:24,2022-06-24 23:08:26
33853,<security>,2020-09-14 14:46:32,2022-06-25 08:46:53


In [18]:
import re
# convert to lowercase
dataset['CreationDate'] = dataset['CreationDate'].str.lower()
dataset['LastActivityDate'] = dataset['LastActivityDate'].str.lower()

# convert desc to string
dataset['CreationDate'] = dataset['CreationDate'].astype(str)
dataset['LastActivityDate'] = dataset['LastActivityDate'].astype(str)

dataset['Start1'] = dataset['CreationDate'].apply(lambda x: ''.join([item for item in x.split('-')]))
dataset['Start2'] = dataset['Start1'].apply(lambda x: ''.join([item for item in x.split(':')]))
dataset['Start3'] = dataset['Start2'].apply(lambda x: ''.join([item for item in x.split(' ')]))
dataset['Start'] = dataset['Start3'].apply(lambda x: ''.join([item for item in x[:-6]]))

dataset['End1'] = dataset['LastActivityDate'].apply(lambda x: ''.join([item for item in x.split('-')]))
dataset['End2'] = dataset['End1'].apply(lambda x: ''.join([item for item in x.split(':')]))
dataset['End3'] = dataset['End2'].apply(lambda x: ''.join([item for item in x.split(' ')]))
dataset['End'] = dataset['End3'].apply(lambda x: ''.join([item for item in x[:-6]]))

dataset.head(10)

,Tags,CreationDate,LastActivityDate,Start1,Start2,Start3,Start,End1,End2,End,End3
0,<amazon-web-services><amazon-ec2><network-secu...,2017-06-21 06:29:34,2017-06-21 13:24:00,20170621 06:29:34,20170621 062934,20170621062934,20170621,20170621 13:24:00,20170621 132400,20170621,20170621132400
1,<amazon-web-services><amazon-ec2><network-secu...,2017-06-21 06:29:34,2017-06-21 13:24:00,20170621 06:29:34,20170621 062934,20170621062934,20170621,20170621 13:24:00,20170621 132400,20170621,20170621132400
2,<javascript><reactjs><security><jwt><rails-api>,2017-06-21 22:30:57,2017-06-21 23:07:12,20170621 22:30:57,20170621 223057,20170621223057,20170621,20170621 23:07:12,20170621 230712,20170621,20170621230712
3,<facebook><azure><security><azure-blob-storage>,2017-06-21 15:28:37,2017-06-22 03:22:55,20170621 15:28:37,20170621 152837,20170621152837,20170621,20170622 03:22:55,20170622 032255,20170622,20170622032255
4,<security>,2017-06-21 14:53:05,2017-06-22 09:55:00,20170621 14:53:05,20170621 145305,20170621145305,20170621,20170622 09:55:00,20170622 095500,20170622,20170622095500
5,<security><certificate><ibm-mq>,2017-06-22 08:06:59,2017-07-12 08:47:07,20170622 08:06:59,20170622 080659,20170622080659,20170622,20170712 08:47:07,20170712 084707,20170712,20170712084707
6,<ios><objective-c><security><encryption><sha1>,2017-06-22 09:20:16,2018-06-24 12:36:11,20170622 09:20:16,20170622 092016,20170622092016,20170622,20180624 12:36:11,20180624 123611,20180624,20180624123611
7,<ios><objective-c><security><encryption><sha1>,2017-06-22 09:20:16,2018-06-24 12:36:11,20170622 09:20:16,20170622 092016,20170622092016,20170622,20180624 12:36:11,20180624 123611,20180624,20180624123611
8,<java><performance><security><rsa><digital-sig...,2017-06-22 14:32:16,2017-06-25 00:22:45,20170622 14:32:16,20170622 143216,20170622143216,20170622,20170625 00:22:45,20170625 002245,20170625,20170625002245
9,<ios><objective-c><security><encryption><sha1>,2017-06-22 09:20:16,2018-06-24 12:36:11,20170622 09:20:16,20170622 092016,20170622092016,20170622,20180624 12:36:11,20180624 123611,20180624,20180624123611


In [19]:
product_headers = ['Tags', 'Start', 'End']
export_dataset = pd.DataFrame(dataset, columns=product_headers)
 
export_dataset
export_dataset.to_csv('Time_cyber.csv', encoding='utf-8', index=False)

In [20]:
# selected column headers we want to use 
new_headers = ['Tags', 'Start', 'End']
df = pd.read_csv("Time_cyber.csv", header=0)
# remove unwanted columns keep selected headers
dataset = pd.DataFrame(df, columns=new_headers)

dataset

,Tags,Start,End
0,<amazon-web-services><amazon-ec2><network-secu...,20170621,20170621
1,<amazon-web-services><amazon-ec2><network-secu...,20170621,20170621
2,<javascript><reactjs><security><jwt><rails-api>,20170621,20170621
3,<facebook><azure><security><azure-blob-storage>,20170621,20170622
4,<security>,20170621,20170622
...,...,...,...
33850,<php><wordpress><security><xss>,20220624,20220624
33851,<android><firebase-realtime-database><firebase...,20220624,20220624
33852,<asp.net-core><security><.net-core><ssl-certif...,20210129,20220624
33853,<security>,20200914,20220625


In [25]:
dataset['TimeDiff'] = dataset['End'] - dataset['Start']

dataset.head(10)

,Tags,Start,End,TimeDiff
0,<amazon-web-services><amazon-ec2><network-secu...,20170621,20170621,0
1,<amazon-web-services><amazon-ec2><network-secu...,20170621,20170621,0
2,<javascript><reactjs><security><jwt><rails-api>,20170621,20170621,0
3,<facebook><azure><security><azure-blob-storage>,20170621,20170622,1
4,<security>,20170621,20170622,1
5,<security><certificate><ibm-mq>,20170622,20170712,90
6,<ios><objective-c><security><encryption><sha1>,20170622,20180624,10002
7,<ios><objective-c><security><encryption><sha1>,20170622,20180624,10002
8,<java><performance><security><rsa><digital-sig...,20170622,20170625,3
9,<ios><objective-c><security><encryption><sha1>,20170622,20180624,10002


In [34]:
dataset.sort_values(by = "TimeDiff", ascending=False)

,Tags,Start,End,TimeDiff
46,<javascript><html><security><content-security-...,20170630,20220614,49984
43,<javascript><html><security><content-security-...,20170630,20220614,49984
365,<java><spring><spring-boot><spring-security><s...,20170622,20220526,49904
433,<java><spring><spring-boot><spring-security><s...,20170622,20220526,49904
32323,<java><spring><spring-boot><spring-security><s...,20170622,20220526,49904
...,...,...,...,...
12092,<java><security><reflection>,20190228,20190228,0
12091,<security><filesystems><usb>,20190301,20190301,0
24361,<firebase><firebase-realtime-database><firebas...,20201230,20201230,0
12090,<linux><amazon-web-services><security><amazon-...,20190303,20190303,0


In [35]:
product_headers = ['Tags', 'Start', 'End', 'TimeDiff']
export_dataset = pd.DataFrame(dataset, columns=product_headers)
 
export_dataset
export_dataset.to_csv('Time_sort.csv', encoding='utf-8', index=False)